In [ ]:
# YOLO11 Training Notebook

This notebook provides a comprehensive interface for training YOLO11 models on custom datasets.
Based on the infrared object detection dataset for person, bicycle, and car detection.

## Features
- Model training with customizable parameters
- Real-time training monitoring
- Model validation and testing
- Results visualization
- Model export capabilities


In [ ]:
## 1. Setup and Imports


In [ ]:
import os
import sys
import time
import logging
from pathlib import Path
from datetime import datetime

import torch
import yaml
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image, clear_output
import ipywidgets as widgets

# Import YOLO11
from ultralytics import YOLO

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")


In [ ]:
## 2. Configuration Setup


In [ ]:
# Project configuration
PROJECT_ROOT = Path.cwd()
DATA_CONFIG = PROJECT_ROOT / 'data_infrared.yaml'
WEIGHTS_PATH = PROJECT_ROOT / 'yolo11n.pt'
RUNS_DIR = PROJECT_ROOT / 'runs' / 'train'

# Training configuration
TRAINING_CONFIG = {
    'model_size': 'yolo11n.pt',  # yolo11n, yolo11s, yolo11m, yolo11l, yolo11x
    'epochs': 100,
    'batch_size': 16,
    'img_size': 640,
    'workers': 8,
    'device': '',  # '' for auto, '0' for GPU 0, 'cpu' for CPU
    'project': 'runs/train',
    'name': 'yolo11_infrared',
    'patience': 50,
    'save_period': 10,
    'cache': 'ram',  # 'ram', 'disk', or False
    'amp': True,  # Automatic Mixed Precision
    'optimizer': 'auto',  # 'SGD', 'Adam', 'AdamW', 'auto'
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'cos_lr': False,
    'rect': False,
    'resume': False
}

print("Configuration loaded:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")


In [ ]:
## 3. Dataset Information


In [ ]:
# Load and display dataset configuration
def load_dataset_config(config_path):
    """Load dataset configuration from YAML file."""
    with open(config_path, 'r') as f:
        return yaml.safe_load(f)

def display_dataset_info(config):
    """Display dataset information."""
    print("Dataset Configuration:")
    print(f"  Path: {config['path']}")
    print(f"  Classes: {config['nc']}")
    print(f"  Class names: {config['names']}")
    print(f"  Train: {config['train']}")
    print(f"  Validation: {config['val']}")
    
    # Check dataset statistics
    dataset_path = Path(config['path'])
    if dataset_path.exists():
        train_images = list((dataset_path / config['train']).glob('*.jpg'))
        val_images = list((dataset_path / config['val']).glob('*.jpg'))
        
        print(f"\nDataset Statistics:")
        print(f"  Training images: {len(train_images)}")
        print(f"  Validation images: {len(val_images)}")
        print(f"  Total images: {len(train_images) + len(val_images)}")
    else:
        print(f"\nWarning: Dataset path {dataset_path} does not exist!")

# Load and display dataset info
if DATA_CONFIG.exists():
    dataset_config = load_dataset_config(DATA_CONFIG)
    display_dataset_info(dataset_config)
else:
    print(f"Warning: Dataset config file {DATA_CONFIG} not found!")


In [ ]:
## 4. Model Setup


In [ ]:
def load_model(weights_path, verbose=True):
    """Load YOLO11 model."""
    try:
        model = YOLO(weights_path)
        if verbose:
            print(f"Model loaded successfully: {weights_path}")
            print(f"Model architecture: {model.model}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

# Load the model
print(f"Loading model: {TRAINING_CONFIG['model_size']}")
model = load_model(TRAINING_CONFIG['model_size'])

if model:
    print("\nModel summary:")
    try:
        model.info()
    except:
        print("Model info not available")


In [ ]:
## 5. Interactive Training Configuration


In [ ]:
# Interactive training configuration widgets
epochs_widget = widgets.IntSlider(value=100, min=1, max=500, description='Epochs:')
batch_size_widget = widgets.IntSlider(value=16, min=1, max=64, description='Batch Size:')
img_size_widget = widgets.Dropdown(options=[320, 416, 512, 640, 832, 1024], value=640, description='Image Size:')
model_widget = widgets.Dropdown(options=['yolo11n.pt', 'yolo11s.pt', 'yolo11m.pt', 'yolo11l.pt', 'yolo11x.pt'], 
                               value='yolo11n.pt', description='Model:')
optimizer_widget = widgets.Dropdown(options=['auto', 'SGD', 'Adam', 'AdamW'], value='auto', description='Optimizer:')
device_widget = widgets.Dropdown(options=['auto', '0', 'cpu'], value='auto', description='Device:')
patience_widget = widgets.IntSlider(value=50, min=10, max=200, description='Patience:')
lr0_widget = widgets.FloatLogSlider(value=0.01, base=10, min=-4, max=-1, description='Learning Rate:')

# Display widgets
config_box = widgets.VBox([
    widgets.HTML("<h3>Training Configuration</h3>"),
    epochs_widget,
    batch_size_widget,
    img_size_widget,
    model_widget,
    optimizer_widget,
    device_widget,
    patience_widget,
    lr0_widget
])

display(config_box)


In [ ]:
## 6. Training Function


In [ ]:
def setup_training_logging(log_dir):
    """Setup logging for training."""
    log_dir = Path(log_dir)
    log_dir.mkdir(parents=True, exist_ok=True)
    
    log_file = log_dir / f'training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    
    return logging.getLogger('YOLO11_TRAIN'), log_file

def train_model(model, config, data_config_path, log_progress=True):
    """Train YOLO11 model with given configuration."""
    
    # Setup logging
    log_dir = Path(config['project']) / config['name']
    logger, log_file = setup_training_logging(log_dir)
    
    logger.info("Starting YOLO11 training...")
    logger.info(f"Configuration: {config}")
    
    # Prepare training arguments
    train_args = {
        'data': str(data_config_path),
        'epochs': config['epochs'],
        'batch': config['batch_size'],
        'imgsz': config['img_size'],
        'device': config['device'] if config['device'] != 'auto' else '',
        'workers': config['workers'],
        'project': config['project'],
        'name': config['name'],
        'exist_ok': True,
        'pretrained': True,
        'optimizer': config['optimizer'],
        'patience': config['patience'],
        'save_period': config['save_period'],
        'cache': config['cache'],
        'amp': config['amp'],
        'lr0': config['lr0'],
        'lrf': config['lrf'],
        'momentum': config['momentum'],
        'weight_decay': config['weight_decay'],
        'cos_lr': config['cos_lr'],
        'rect': config['rect'],
        'resume': config['resume'],
        'plots': True,
        'verbose': True
    }
    
    start_time = time.time()
    
    try:
        # Start training
        results = model.train(**train_args)
        
        # Calculate training time
        end_time = time.time()
        training_time = end_time - start_time
        hours = int(training_time // 3600)
        minutes = int((training_time % 3600) // 60)
        seconds = int(training_time % 60)
        
        logger.info(f"Training completed successfully!")
        logger.info(f"Training time: {hours:02d}:{minutes:02d}:{seconds:02d}")
        logger.info(f"Log file: {log_file}")
        
        return results, log_file
        
    except KeyboardInterrupt:
        logger.info("Training interrupted by user")
        return None, log_file
    except Exception as e:
        logger.error(f"Training failed: {e}")
        raise e

print("Training functions defined successfully!")


In [ ]:
## 7. Start Training


In [ ]:
# Update configuration from widgets
TRAINING_CONFIG.update({
    'epochs': epochs_widget.value,
    'batch_size': batch_size_widget.value,
    'img_size': img_size_widget.value,
    'model_size': model_widget.value,
    'optimizer': optimizer_widget.value,
    'device': device_widget.value if device_widget.value != 'auto' else '',
    'patience': patience_widget.value,
    'lr0': lr0_widget.value
})

print("Updated training configuration:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

# Load model with updated configuration
if TRAINING_CONFIG['model_size'] != model.model_name if hasattr(model, 'model_name') else '':
    print(f"\nLoading new model: {TRAINING_CONFIG['model_size']}")
    model = load_model(TRAINING_CONFIG['model_size'])

# Confirm before starting training
print("\n" + "="*50)
print("READY TO START TRAINING")
print("="*50)
print(f"Dataset: {DATA_CONFIG}")
print(f"Model: {TRAINING_CONFIG['model_size']}")
print(f"Epochs: {TRAINING_CONFIG['epochs']}")
print(f"Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"Image size: {TRAINING_CONFIG['img_size']}")
print("="*50)


In [ ]:
# Start training (uncomment to run)
# WARNING: This will start training - make sure your configuration is correct!

# results, log_file = train_model(model, TRAINING_CONFIG, DATA_CONFIG)
# print(f"Training completed! Log file: {log_file}")

print("Uncomment the lines above to start training!")
print("Make sure your configuration is correct before starting.")


In [ ]:
## 8. Training Monitoring and Visualization


In [ ]:
def plot_training_results(results_dir):
    """Plot training results from results directory."""
    results_dir = Path(results_dir)
    
    # Look for results.png or results.csv
    results_png = results_dir / 'results.png'
    results_csv = results_dir / 'results.csv'
    
    if results_png.exists():
        print("Training Results:")
        display(Image(str(results_png)))
    
    if results_csv.exists():
        import pandas as pd
        df = pd.read_csv(results_csv)
        print("\nTraining Metrics:")
        print(df.tail(10))  # Show last 10 epochs
        
        # Plot key metrics
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot losses
        if 'train/box_loss' in df.columns:
            axes[0, 0].plot(df['epoch'], df['train/box_loss'], label='Box Loss')
            axes[0, 0].plot(df['epoch'], df['train/cls_loss'], label='Class Loss')
            axes[0, 0].plot(df['epoch'], df['train/dfl_loss'], label='DFL Loss')
            axes[0, 0].set_title('Training Losses')
            axes[0, 0].legend()
            axes[0, 0].grid(True)
        
        # Plot mAP
        if 'metrics/mAP50(B)' in df.columns:
            axes[0, 1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@0.5')
            axes[0, 1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95')
            axes[0, 1].set_title('Validation mAP')
            axes[0, 1].legend()
            axes[0, 1].grid(True)
        
        # Plot precision and recall
        if 'metrics/precision(B)' in df.columns:
            axes[1, 0].plot(df['epoch'], df['metrics/precision(B)'], label='Precision')
            axes[1, 0].plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
            axes[1, 0].set_title('Precision & Recall')
            axes[1, 0].legend()
            axes[1, 0].grid(True)
        
        # Plot learning rate
        if 'lr/pg0' in df.columns:
            axes[1, 1].plot(df['epoch'], df['lr/pg0'], label='Learning Rate')
            axes[1, 1].set_title('Learning Rate')
            axes[1, 1].legend()
            axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.show()

# Example usage (uncomment and adjust path)
# results_dir = 'runs/train/yolo11_infrared'
# plot_training_results(results_dir)

print("Training monitoring functions ready!")
print("Use plot_training_results('path/to/results') to visualize training progress.")


In [ ]:
## 9. Model Validation and Testing


In [ ]:
def validate_model(model_path, data_config, img_size=640):
    """Validate trained model."""
    model = YOLO(model_path)
    
    print(f"Validating model: {model_path}")
    print(f"Dataset: {data_config}")
    
    # Run validation
    results = model.val(
        data=data_config,
        imgsz=img_size,
        save_json=True,
        save_hybrid=True,
        plots=True
    )
    
    print("\nValidation Results:")
    print(f"mAP@0.5: {results.box.map50:.4f}")
    print(f"mAP@0.5:0.95: {results.box.map:.4f}")
    print(f"Precision: {results.box.mp:.4f}")
    print(f"Recall: {results.box.mr:.4f}")
    
    return results

def test_inference(model_path, image_path, conf_threshold=0.25):
    """Test model inference on a single image."""
    model = YOLO(model_path)
    
    # Run inference
    results = model(image_path, conf=conf_threshold)
    
    # Display results
    for r in results:
        # Plot results
        im_array = r.plot()
        
        # Convert BGR to RGB for matplotlib
        im_array = im_array[:, :, ::-1]
        
        plt.figure(figsize=(12, 8))
        plt.imshow(im_array)
        plt.axis('off')
        plt.title(f'Inference Results - Confidence: {conf_threshold}')
        plt.show()
        
        # Print detection info
        if r.boxes is not None:
            print(f"Detections: {len(r.boxes)}")
            for i, box in enumerate(r.boxes):
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                print(f"  {i+1}: Class {cls} ({model.names[cls]}) - Confidence: {conf:.3f}")
        else:
            print("No detections found")
    
    return results

print("Validation functions ready!")
print("Use validate_model() and test_inference() to test your trained model.")


In [ ]:
## 10. Model Export


In [ ]:
def export_model(model_path, export_formats=['onnx'], img_size=640):
    """Export trained model to different formats."""
    model = YOLO(model_path)
    
    print(f"Exporting model: {model_path}")
    
    exported_models = {}
    
    for fmt in export_formats:
        try:
            print(f"Exporting to {fmt.upper()}...")
            exported_path = model.export(
                format=fmt,
                imgsz=img_size,
                optimize=True,
                half=False
            )
            exported_models[fmt] = exported_path
            print(f"  ✓ Exported to: {exported_path}")
        except Exception as e:
            print(f"  ✗ Failed to export to {fmt}: {e}")
    
    return exported_models

# Available export formats
EXPORT_FORMATS = [
    'onnx',      # ONNX
    'torchscript', # TorchScript
    'tensorflow', # TensorFlow
    'tflite',    # TensorFlow Lite
    'edgetpu',   # Edge TPU
    'openvino',  # OpenVINO
    'coreml',    # CoreML
    'tensorrt'   # TensorRT
]

print("Export functions ready!")
print(f"Available formats: {EXPORT_FORMATS}")
print("Use export_model() to convert your trained model.")


In [ ]:
## 11. Quick Start Examples


In [ ]:
# Quick start examples - uncomment to use

# Example 1: Validate a trained model
# model_path = 'runs/train/yolo11_infrared/weights/best.pt'
# results = validate_model(model_path, DATA_CONFIG)

# Example 2: Test inference on an image
# test_image = 'datasets/infrared/images/val/1.jpg'
# inference_results = test_inference(model_path, test_image, conf_threshold=0.5)

# Example 3: Export model to ONNX
# exported = export_model(model_path, ['onnx'], img_size=640)

# Example 4: Plot training results
# plot_training_results('runs/train/yolo11_infrared')

print("Quick start examples ready!")
print("Uncomment the examples above to use them.")


In [ ]:
## 12. Utilities and System Information


In [ ]:
def list_available_models():
    """List available YOLO11 models."""
    models = {
        'yolo11n.pt': 'Nano - Fastest, smallest model',
        'yolo11s.pt': 'Small - Good balance of speed and accuracy',
        'yolo11m.pt': 'Medium - Better accuracy, slower',
        'yolo11l.pt': 'Large - High accuracy, slower',
        'yolo11x.pt': 'Extra Large - Highest accuracy, slowest'
    }
    
    print("Available YOLO11 models:")
    for model, description in models.items():
        print(f"  {model}: {description}")
    
    return models

def check_dataset_integrity(dataset_config_path):
    """Check dataset integrity and statistics."""
    config = load_dataset_config(dataset_config_path)
    dataset_path = Path(config['path'])
    
    print("Dataset Integrity Check:")
    print(f"Dataset path: {dataset_path}")
    
    # Check directories
    train_img_dir = dataset_path / config['train']
    val_img_dir = dataset_path / config['val']
    train_lbl_dir = dataset_path / 'labels' / 'train'
    val_lbl_dir = dataset_path / 'labels' / 'val'
    
    dirs_to_check = {
        'Train images': train_img_dir,
        'Val images': val_img_dir,
        'Train labels': train_lbl_dir,
        'Val labels': val_lbl_dir
    }
    
    for name, path in dirs_to_check.items():
        if path.exists():
            count = len(list(path.glob('*')))
            print(f"  ✓ {name}: {count} files")
        else:
            print(f"  ✗ {name}: Directory not found")
    
    return config

def get_system_info():
    """Get system information for debugging."""
    info = {
        'Python version': sys.version,
        'PyTorch version': torch.__version__,
        'CUDA available': torch.cuda.is_available(),
        'GPU count': torch.cuda.device_count() if torch.cuda.is_available() else 0,
    }
    
    if torch.cuda.is_available():
        info['GPU name'] = torch.cuda.get_device_name(0)
        info['CUDA version'] = torch.version.cuda
        info['GPU memory'] = f"{torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB"
    
    print("System Information:")
    for key, value in info.items():
        print(f"  {key}: {value}")
    
    return info

# Run utility functions
list_available_models()
print()
get_system_info()
print()
if DATA_CONFIG.exists():
    check_dataset_integrity(DATA_CONFIG)
